# Generic Math

- `static virtual` members in interfaces
- `static abstract` members in interfaces
- `checked` user-defined operators
- Unsigned right-shift operator
- Relaxed shift operator

See also "curiously recurring template pattern" (CRTP)

In [ ]:
MyMain();

public interface IMyNumber<TSelf>
    where TSelf : IMyNumber<TSelf>
{
    static abstract TSelf Zero { get; }
    static abstract TSelf One { get; }
    static abstract TSelf MinusOne { get; }

    static abstract TSelf operator +(TSelf left, TSelf right);
    static virtual TSelf operator checked +(TSelf left, TSelf right) => left + right;
}

public readonly struct MyNumber : IMyNumber<MyNumber>
{
    private readonly sbyte _value;

    public MyNumber(sbyte value) => _value = value;

    public static MyNumber Zero => new MyNumber();
    public static MyNumber One => new MyNumber(1);
    public static MyNumber MinusOne => new MyNumber(-1);

    public static MyNumber operator +(MyNumber left, MyNumber right) => new MyNumber((sbyte)(left._value + right._value));
    public static MyNumber operator checked +(MyNumber left, MyNumber right) => new MyNumber(checked((sbyte)(left._value + right._value)));

    public static MyNumber operator <<(MyNumber value, MyNumber shiftAmount) => new MyNumber((sbyte)(value._value << shiftAmount._value));
    public static MyNumber operator >>(MyNumber value, MyNumber shiftAmount) => new MyNumber((sbyte)(value._value >> shiftAmount._value));
    public static MyNumber operator >>>(MyNumber value, int shiftAmount) => new MyNumber((sbyte)(value._value >>> shiftAmount));

    public override string ToString() => _value.ToString();
    public string ToBinaryString() => Convert.ToString(_value, 2).PadLeft(8, '0');
}

public static class MyMath
{
    public static TResult Sum<TResult>(ReadOnlySpan<TResult> values)
        where TResult : struct, IMyNumber<TResult>
    {
        TResult sum = TResult.Zero;
        foreach (TResult value in values)
        {
            checked { sum += value; }
        }
        return sum;
    }
}

public void MyMain()
{
    Span<MyNumber> span = stackalloc MyNumber[] { new MyNumber(100), new MyNumber(20), new MyNumber(7) };

    MyNumber sum = MyMath.Sum<MyNumber>(span);

    Console.WriteLine(sum);
    Console.WriteLine((sum >>> 0).ToBinaryString());
}